In [1]:
from fastplotlib.widgets import ImageWidget
import numpy as np
import tifffile

# Single image sequence

In [2]:
# using Sue demo movie
path = "/home/kushal/caiman_data/example_movies/Sue_2x_3000_40_-46.tif"
a = tifffile.imread(path)

In [3]:
iw = ImageWidget(
    data=a, 
    vmin_vmax_sliders=True,
    cmap="gnuplot2"
)

RFBOutputContext()

MESA-INTEL: warning: Performance support disabled, consider sysctl dev.i915.perf_stream_paranoid=0

Disabling presentation on 'Intel(R) UHD Graphics 620 (KBL GT2)' (id 0x41804f0) because of NV Optimus (on Linux)


In [4]:
iw.show()

# Try scrolling along the "t" - time dimension and changing the min-max!

### Play with setting different window functions

These can also be given as kwargs to `ImageWidget` during instantiation

In [12]:
# must be in the form of {dim: (func, window_size)}
iw.window_funcs = {"t": (np.mean, 5)}

In [13]:
# change the winow size
iw.window_funcs["t"].window_size = 23

In [7]:
# change the function
iw.window_funcs["t"].func = np.max

In [8]:
# or set it again
iw.window_funcs = {"t": (np.min, 11)}

### `ImageWidget` embeds a `Plot` or `GridPlot` that is accessible using `ImageWidget.plot`

In [9]:
iw.plot

unnamed: Plot @ 0x7ff2277d5bb0
  parent: None
  Graphics:
	'image': ImageGraphic @ 0x7ff1f8803520

# graphics etc. just like before

In [10]:
iw.plot.graphics

('image': ImageGraphic @ 0x7ff1f8803520,)

In [11]:
iw.plot.graphics[0].cmap="gray"

# Close the canvas

In [14]:
iw.plot.canvas.close()

# Gridplot of txy data

In [15]:
dims = (100, 512, 512)
data = [np.random.rand(*dims) for i in range(4)]

In [16]:
iw = ImageWidget(
    data=data, 
    vmin_vmax_sliders=True,
    names=["zero", "one", "two", "three"],
    window_funcs={"t": (np.mean, 5)},  # you can pass window_funcs as a kwarg
    cmap="gnuplot2", 
)

RFBOutputContext()

### pan-zoom controllers are all synced in a `ImageWidget`

In [17]:
iw.show()

### This `ImageWidget` contains a `GridPlot`, so you can index the subplots using the names given to `ImageWidget`

In [18]:
iw.plot["two"]

two: Subplot @ 0x7ff194364f40
  parent: None
  Graphics:
	'image': ImageGraphic @ 0x7ff1942f7f40

### change window functions just like before

In [19]:
iw.window_funcs["t"].func = np.max

In [20]:
# make sure to close plot before moving to the next one
iw.plot.canvas.close()

## Gridplot of volumetric data

In [21]:
dims = (256, 256, 5, 100)
data = [np.random.rand(*dims) for i in range(4)]

iw = ImageWidget(
    data=data, 
    slider_dims=["t", "z"], 
    dims_order="xyzt", # example of how you can set this for non-standard orders
    vmin_vmax_sliders=True,
    names=["zero", "one", "two", "three"],
    # window_funcs={"t": (np.mean, 5)}, # window functions can be slow when indexing multiple dims
    cmap="gnuplot2", 
)

RFBOutputContext()

In [22]:
iw.show()

### window functions, can be slow when you have "t" and "z"

In [23]:
iw.window_funcs = {"t": (np.mean, 11)}

In [24]:
# make sure to close plot
iw.plot.canvas.close()

# Volumetric zebrafish data, view each plane using a `GridPlot`

#### `tifffile.memmap` is a nice way to lazy-load large imaging files. This file is kinda small (10G), but we can load it instantly using `tifffile.memmap`, it would take ~@0 seconds to load into RAM on this laptop with a SATA SSD.

In [25]:
zfish = tifffile.memmap("/home/kushal/Downloads/zfish_vol.tiff")

In [26]:
zfish.shape

(1320, 30, 512, 512)

In [27]:
iw = ImageWidget(zfish, vmin_vmax_sliders=True, cmap="gnuplot2")
iw.show()

RFBOutputContext()

In [28]:
iw.window_funcs = {"t": (np.mean, 5), "z": None}

In [29]:
iw.window_funcs = {"t": (np.mean, 5), "z": (np.mean, 10)}

In [30]:
zfish.shape

(1320, 30, 512, 512)

In [31]:
zfish_split = [zfish[:, i, :, :] for i in range(zfish.shape[1])]

In [32]:
iw = ImageWidget(zfish_split, window_funcs={"t": (np.mean, 5)}, cmap="gnuplot2")
iw.show()

RFBOutputContext()

In [33]:
from mesmerize_core.arrays import LazyTiff

2023-01-30 10:17:07.338867: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-30 10:17:07.471128: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-30 10:17:07.471160: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-30 10:17:08.155125: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'

In [34]:
ll -h /home/kushal/Downloads/20220513_rec1.tif

-rw-r--r-- 1 kushal 39G Jan 23 17:15 /home/kushal/Downloads/20220513_rec1.tif


In [35]:
movie = LazyTiff("/home/kushal/Downloads/20220513_rec1.tif")

In [36]:
iw = ImageWidget(movie, vmin_vmax_sliders=True, cmap="gnuplot2")
iw.show()

RFBOutputContext()

/home/kushal/venvs/workshop/lib/python3.9/site-packages/mesmerize_core/arrays/_tiff.py:39: UserWarning: min not implemented for LazyTiff, returning min of 0th index
  warn("min not implemented for LazyTiff, returning min of 0th index")
/home/kushal/venvs/workshop/lib/python3.9/site-packages/mesmerize_core/arrays/_tiff.py:44: UserWarning: max not implemented for LazyTiff, returning min of 0th index
  warn("max not implemented for LazyTiff, returning min of 0th index")


In [37]:
# workaround for now until there's a permanent fix with LazyTiff
# use frame_apply and set the window averaging manually.
window_size = 23
flank = int((window_size - 1) / 2)

iw.frame_apply = {0: lambda f: np.mean(movie[iw.current_index["t"] - flank:iw.current_index["t"] + flank], axis=0)}

# Kill the kernel before moving on to the next demo!